In [6]:
# !pip install langchain boto3

## 기본환경(library, Class, Functions) 설정

In [10]:
from langchain.embeddings import BedrockEmbeddings
from numpy import dot
from numpy.linalg import norm

#Bedrock Embeddings LangChain 클라이언트 생성
#https://api.python.langchain.com/en/latest/embeddings/langchain_community.embeddings.bedrock.BedrockEmbeddings.html
belc = BedrockEmbeddings()

class EmbedItem:
    def __init__(self, text):
        self.text = text
        self.embedding = belc.embed_query(text)

class ComparisonResult:
    def __init__(self, text, similarity):
        self.text = text
        self.similarity = similarity

def calculate_similarity(a, b): # 코사인 유사도를 확인하세요: https://en.wikipedia.org/wiki/Cosine_similarity
    return dot(a, b) / (norm(a) * norm(b))


## 임베딩할 Text 목록을 읽어오며, Titan Eebedding 값 추가

In [8]:
#비교할 임베딩 목록을 작성합니다.
items = []

with open("items.txt", "r") as f:
    text_items = f.read().splitlines()

for text in text_items:
    items.append(EmbedItem(text))


## 모든 Line의 Text를 서로간의 코사인 유사도를 비교

In [11]:
for e1 in items:
    print(f"Closest matches for '{e1.text}'")
    print ("----------------")
    cosine_comparisons = []
    
    for e2 in items:
        similarity_score = calculate_similarity(e1.embedding, e2.embedding) # 두 문장간 코사인 유사도를 구하고
        
        cosine_comparisons.append(ComparisonResult(e2.text, similarity_score)) # 코사인 유사도 값 을 목록에 저장합니다.
        
    cosine_comparisons.sort(key=lambda x: x.similarity, reverse=True) # 가장 가까운 일치 항목을 먼저 나열합니다.
    
    for c in cosine_comparisons:
        print("%.6f" % c.similarity, "\t", c.text)
    
    print()

Closest matches for 'Felines, canines, and rodents'
----------------
1.000000 	 Felines, canines, and rodents
0.872856 	 Cats, dogs, and mice
0.599730 	 Chats, chiens et souris
0.516598 	 Lions, tigers, and bears
0.456268 	 고양이, 개, 쥐
0.455923 	 猫、犬、ネズミ
0.068916 	 パン屋への道順を知りたい
0.061314 	 パン屋への行き方を教えてください
0.034925 	 빵집으로 가는 길을 알려주세요.
0.024160 	 경기장 가는 방법을 알려주시겠어요?
0.002239 	 Can you please tell me how to get to the stadium?
-0.003159 	 Kannst du mir bitte sagen, wie ich zur Bäckerei komme?
-0.007595 	 Can you please tell me how to get to the bakery?
-0.019469 	 Pouvez-vous s'il vous plaît me dire comment me rendre à la boulangerie?
-0.020840 	 I need directions to the bread shop

Closest matches for 'Can you please tell me how to get to the bakery?'
----------------
1.000000 	 Can you please tell me how to get to the bakery?
0.712236 	 I need directions to the bread shop
0.541959 	 Pouvez-vous s'il vous plaît me dire comment me rendre à la boulangerie?
0.492384 	 빵집으로 가는 길을 알려주세요.
0.4846